In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
from torchvision import datasets,transforms 
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np 
import torch 

num_workers = 0
batch_sizes = 32
valid_size = 0.2 

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
traindata = datasets.CIFAR10(root='CIFAR_10', download=True, train=True, transform = train_transform)
testdata = datasets.CIFAR10(root='CIFAR_10', download=True, train=False, transform = test_transform)

num_train = len(traindata)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(num_train*valid_size))
train_idx, valid_idx = indices[split:], indices[:split]
# print(train_idx)
train_sampler, valid_sampler = SubsetRandomSampler(train_idx), SubsetRandomSampler(valid_idx)

trainloader = torch.utils.data.DataLoader(traindata, sampler=train_sampler, batch_size=batch_sizes, num_workers=num_workers)
validloader = torch.utils.data.DataLoader(traindata, sampler=valid_sampler, batch_size=batch_sizes, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(testdata, batch_size=batch_sizes, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
import torch.nn as nn 
import torch.nn.functional as F

class Net(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)

    self.fc1 = nn.Linear(64*4*4, 500)
    self.fc2 = nn.Linear(500, 10)
    self.dropout = nn.Dropout(0.25)
  
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 64*4*4)
    x = self.dropout(x)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
  
    return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = Net().to(device)
model


cuda:0


Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [0]:
import torch.optim as optim 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
EPOCHS = 30

valid_loss_min = np.Inf 

for epoch in range(EPOCHS):

  train_loss = 0.0
  valid_loss = 0.0

  model.train()
  for images, labels in trainloader:
    
    optimizer.zero_grad()
    # print(images.shape)
    images, labels = images.to(device), labels.to(device)
    outputs = model(images).to(device)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
  
  model.eval()
  with torch.no_grad():

    for images, labels in validloader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images).to(device)
      loss = criterion(outputs, labels)
      valid_loss += loss.item()
  
  train_loss = train_loss/len(trainloader)
  valid_loss = valid_loss/len(validloader)

  print('EPOCH : {}\t Training loss : {:.3f} \t Validation loss : {:.3f}. '.format(epoch+1, train_loss, valid_loss))

  if valid_loss < valid_loss_min:
    print('Validation loss decreased ({:.3f} --> {:.3f}) . Saving model ...\n'.format(valid_loss_min, valid_loss))
    valid_loss_min = valid_loss 
    torch.save(model.state_dict(), 'model.pt')
  



EPOCH : 1	 Training loss : 2.220 	 Validation loss : 2.029. 
Validation loss decreased (inf --> 2.029) . Saving model ...

EPOCH : 2	 Training loss : 1.909 	 Validation loss : 1.733. 
Validation loss decreased (2.029 --> 1.733) . Saving model ...

EPOCH : 3	 Training loss : 1.654 	 Validation loss : 1.561. 
Validation loss decreased (1.733 --> 1.561) . Saving model ...

EPOCH : 4	 Training loss : 1.525 	 Validation loss : 1.468. 
Validation loss decreased (1.561 --> 1.468) . Saving model ...

EPOCH : 5	 Training loss : 1.441 	 Validation loss : 1.391. 
Validation loss decreased (1.468 --> 1.391) . Saving model ...

EPOCH : 6	 Training loss : 1.374 	 Validation loss : 1.327. 
Validation loss decreased (1.391 --> 1.327) . Saving model ...

EPOCH : 7	 Training loss : 1.321 	 Validation loss : 1.271. 
Validation loss decreased (1.327 --> 1.271) . Saving model ...

EPOCH : 8	 Training loss : 1.270 	 Validation loss : 1.204. 
Validation loss decreased (1.271 --> 1.204) . Saving model ...

EP

In [0]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
acc = 0.0
true = torch.zeros((10))
total = torch.zeros_like(true)

for images, labels in testloader:
  with torch.no_grad():
    images, labels = images.to(device), labels.to(device)
    outputs = model(images).to(device)
    loss = criterion(outputs, labels)
    
    top_k, top_label = outputs.topk(1, dim=1)

    equal = top_label == labels.view(*top_label.shape) 
    for i in range(len(equal)):
      total[labels[i]] += 1
      if equal[i]:
        true[labels[i]] += 1
    acc += torch.mean(equal.type(torch.float))


print("Accuracy : %.2f%%." %(100*acc/len(testloader)))
for i in range(10):
  print("Accuracy %5s : %.2f%%." %(classes[i], 100*true[i]/total[i]))


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Accuracy : 73.34%.
Accuracy airplane : 74.10%.
Accuracy automobile : 84.50%.
Accuracy  bird : 56.50%.
Accuracy   cat : 50.60%.
Accuracy  deer : 73.50%.
Accuracy   dog : 55.60%.
Accuracy  frog : 89.20%.
Accuracy horse : 82.50%.
Accuracy  ship : 87.40%.
Accuracy truck : 79.30%.


In [10]:
import torch.optim as optim 

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
EPOCHS = 30

valid_loss_min = np.Inf 

for epoch in range(EPOCHS):

  train_loss = 0.0
  valid_loss = 0.0

  model.train()
  for images, labels in trainloader:
    
    optimizer.zero_grad()
    # print(images.shape)
    images, labels = images.to(device), labels.to(device)
    outputs = model(images).to(device)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
  
  model.eval()
  with torch.no_grad():

    for images, labels in validloader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images).to(device)
      loss = criterion(outputs, labels)
      valid_loss += loss.item()
  
  train_loss = train_loss/len(trainloader)
  valid_loss = valid_loss/len(validloader)

  print('EPOCH : {}\t Training loss : {:.3f} \t Validation loss : {:.3f}. '.format(epoch+1, train_loss, valid_loss))

  if valid_loss < valid_loss_min:
    print('Validation loss decreased ({:.3f} --> {:.3f}) . Saving model ...\n'.format(valid_loss_min, valid_loss))
    valid_loss_min = valid_loss 
    torch.save(model.state_dict(), 'model.pt')
  



EPOCH : 1	 Training loss : 1.561 	 Validation loss : 1.315. 
Validation loss decreased (inf --> 1.315) . Saving model ...

EPOCH : 2	 Training loss : 1.346 	 Validation loss : 1.219. 
Validation loss decreased (1.315 --> 1.219) . Saving model ...

EPOCH : 3	 Training loss : 1.269 	 Validation loss : 1.173. 
Validation loss decreased (1.219 --> 1.173) . Saving model ...

EPOCH : 4	 Training loss : 1.239 	 Validation loss : 1.154. 
Validation loss decreased (1.173 --> 1.154) . Saving model ...

EPOCH : 5	 Training loss : 1.212 	 Validation loss : 1.148. 
Validation loss decreased (1.154 --> 1.148) . Saving model ...

EPOCH : 6	 Training loss : 1.197 	 Validation loss : 1.119. 
Validation loss decreased (1.148 --> 1.119) . Saving model ...

EPOCH : 7	 Training loss : 1.170 	 Validation loss : 1.068. 
Validation loss decreased (1.119 --> 1.068) . Saving model ...

EPOCH : 8	 Training loss : 1.169 	 Validation loss : 1.063. 
Validation loss decreased (1.068 --> 1.063) . Saving model ...

EP

In [0]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [11]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
acc = 0.0
true = torch.zeros((10))
total = torch.zeros_like(true)

for images, labels in testloader:
  with torch.no_grad():
    images, labels = images.to(device), labels.to(device)
    outputs = model(images).to(device)
    loss = criterion(outputs, labels)
    
    top_k, top_label = outputs.topk(1, dim=1)

    equal = top_label == labels.view(*top_label.shape) 
    for i in range(len(equal)):
      total[labels[i]] += 1
      if equal[i]:
        true[labels[i]] += 1
    acc += torch.mean(equal.type(torch.float))


print("Accuracy : %.2f%%." %(100*acc/len(testloader)))
for i in range(10):
  print("Accuracy %5s : %.2f%%." %(classes[i], 100*true[i]/total[i]))


Accuracy : 67.13%.
Accuracy airplane : 76.00%.
Accuracy automobile : 83.20%.
Accuracy  bird : 57.60%.
Accuracy   cat : 51.30%.
Accuracy  deer : 66.30%.
Accuracy   dog : 46.70%.
Accuracy  frog : 72.70%.
Accuracy horse : 72.90%.
Accuracy  ship : 74.00%.
Accuracy truck : 70.60%.
